In [0]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import array
from newspaper import Article

In [0]:
app = Flask(__name__)
CORS(app)

In [0]:
def read():
    data = pd.read_csv("C:/Users/JAS/Desktop/Dataset/newsrecomm2new.csv")
    if 'Unnamed: 0' in data.columns:
        data = data.drop('Unnamed: 0', axis=1)
    data.set_index('url',inplace=True)
    return data

In [0]:
def l1l2_calc(ucurr, i, tuples, news_dataset):
    l1l2 = 0
    x = 0
    for x in range(tuples-1):
        if( (news_dataset[ucurr][x] or news_dataset[ucurr][x] == '10') and ( news_dataset[i][x] == 10 or news_dataset[i][x] == '10')  ):
            l1l2 = l1l2 + 1
    return l1l2       
       

def d1d2_calc(ucurr, i, tuples, news_dataset):
    d1d2 = 0
    
    for x in range(tuples-1):
        if( (news_dataset[ucurr][x] == 0 or news_dataset[ucurr][x] == '0' ) and ( news_dataset[i][x] ==0  or news_dataset[i][x] == '0') ):
            d1d2 = d1d2 + 1
    return d1d2



def l1d2_calc(ucurr, i, tuples, news_dataset):
    l1d2 = 0
    for x in range(tuples-1):
        if( (news_dataset[ucurr][x] == 10 or news_dataset[ucurr][x] == '10') and (news_dataset[i][x] == 0 or news_dataset[i][x] == '0') ):
            l1d2 = l1d2 + 1
    return l1d2 

def l2d1_calc(ucurr, i, tuples, news_dataset):
    l2d1 = 0
    for x in range(tuples-1):
        if( (news_dataset[ucurr][x] == 0 or news_dataset[ucurr][x] == '0') and (news_dataset[i][x] == 10 or news_dataset[i][x] == '10') ):
            l2d1 = l2d1 + 1
    return l2d1 


def l1l2d1d2_calc(ucurr, i, tuples, news_dataset):
    l1l2d1d2 = 0
    for x in range(tuples-1):
        if( news_dataset[ucurr][x] != ''  or news_dataset[i][x] != ''):
            l1l2d1d2 = l1l2d1d2 + 1
    return l1l2d1d2




def sim_index_calc(ucurr, i, tuples, news_dataset):
    
    simlarity = 0
    l1l2 = l1l2_calc(ucurr, i, tuples, news_dataset)
    d1d2 = d1d2_calc(ucurr, i, tuples, news_dataset)
    l1d2 = l1d2_calc(ucurr, i, tuples, news_dataset)
    l2d1 = l2d1_calc(ucurr, i, tuples, news_dataset)
    l1l2d1d2 = l1l2d1d2_calc(ucurr, i, tuples, news_dataset)
    similarity = (l1l2 + d1d2 - l1d2 - l2d1 )/ l1l2d1d2
    return similarity

In [0]:
def ZL_calc(ucurr, article_row, news_dataset, n, m, similarity_list):
    sum = 0
    for i in range(1, n+1):
        column = 'user' + str(i)
        email_id = news_dataset.columns.get_values()[i-1]
        
        if((news_dataset[email_id][article_row] == 10 or news_dataset[email_id][article_row] == '10')and ucurr != email_id):
            for j in range(m):
                if(email_id == similarity_list[j]):
                    sum = sum + similarity_list[j+1]
    return sum              
                    
                    
def ZD_calc(ucurr, article_row, news_dataset, n, m, similarity_list):
    sum = 0
    for i in range(1, n+1):
        column = 'user' + str(i)
        email_id = news_dataset.columns.get_values()[i-1]
        
        if((news_dataset[email_id][article_row] == 0 or news_dataset[email_id][article_row] == '0')  and ucurr != email_id ):
            for j in range(m):
                if(email_id == similarity_list[j]):
                    sum = sum + similarity_list[j+1]   
    return sum


def NL(article_row, news_dataset, n):
    nl=0
    for i in range(1,n+1):
        column = 'user' + str(i)
        email_id = news_dataset.columns.get_values()[i-1]
        if(news_dataset[email_id][article_row] == 10 or news_dataset[email_id][article_row] == '10'):
            nl = nl + 1
    return nl


def ND(article_row, news_dataset, n):
    nd = 0
    for i in range(1, n+1):
        column = 'user' + str(i)
        email_id = news_dataset.columns.get_values()[i-1]
        if(news_dataset[email_id][article_row] == 0 or news_dataset[email_id][article_row] == '0'):
            nd = nd + 1
    return nd

def prob_liking_article(ucurr, article_row, news_dataset, n, m, similarity_list):
    ZL_value =  ZL_calc(ucurr, article_row, news_dataset, n, m, similarity_list)
    ZD_value = ZD_calc(ucurr, article_row, news_dataset, n, m, similarity_list)
    NL_value = NL(article_row, news_dataset, n)
    ND_value = ND(article_row, news_dataset, n)
    if (NL_value+ND_value!=0):
        probability = (ZL_value - ZD_value) / (NL_value + ND_value)
    else:
        probability=0;
    return probability

In [0]:
def checkifseen(ucurr, pos, news_dataset):
    row = int( pos/2)
    if(news_dataset[ucurr][row] == 0 or news_dataset[ucurr][row] == 10):
        return True
    else:
        return False

In [0]:
@app.route('/',methods=['POST'])
def action():
    emailData = request.get_json(force=True)
    ucurr = emailData["email"]
    news_dataset=read()
    tuples = len(news_dataset.index)
    n = len(news_dataset.columns)
    similarity_index = array.array('f', [])
    similarity_list = []
    
    for i in range (1,n+1):
        i1 = 'user' + str(i)
        email_id = news_dataset.columns.get_values()[i-1]
        if email_id != ucurr:
            similarity = sim_index_calc( ucurr, email_id, tuples, news_dataset)
            similarity_index.append(similarity)
            similarity_list.append(email_id)
            similarity_list.append(similarity)
        else:
            similarity_list.append(email_id)
            similarity_list.append('ucurr')
    
    m = len(similarity_list)
    # probability array stores values
    prob_array = array.array('f', [])
    probability_list = []
    
    for i in range(tuples):
        probability_value = prob_liking_article(ucurr, i, news_dataset, n, m, similarity_list)
        prob_array.append(probability_value)
        article = news_dataset.index[i]
        probability_list.append(article)
        probability_list.append(probability_value)
        
    recommended_articles =[]
    #print(probability_list)
    j =0
    while j < 5:
        max1 = probability_list[1]
        url=''
        pos =0
        for i in range(2, len(probability_list)):
            if(i%2!=0):
                if(probability_list[i] > max1):
                    max1 = probability_list[i]
                    url = probability_list[i-1]
                    pos =i
                    
        probability_list.pop(pos)
        probability_list.pop(pos-1)  
        seen = checkifseen(ucurr, pos, news_dataset)
        if(seen == False):
            j = j + 1
            recommended_articles.append(url)
    recom=[]
    for Url in recommended_articles:
        article = Article(Url)
        article.download()
        article.parse()
        recom.append(Url)
        recom.append(article.top_image)
        recom.append(article.title)
        recom.append(Url.split("//")[-1].split("/")[0])
        recom.append(article.text[:125]+"...")
        
    return jsonify({"url1": recom[0], "img1" : recom[1],"title1" : recom[2], "source1": recom[3],"summary1" : recom[4],
                   "url2": recom[5], "img2" : recom[6],"title2" : recom[7], "source2": recom[8],"summary2" : recom[9],
                   "url3": recom[10], "img3" : recom[11],"title3" : recom[12], "source3": recom[13],"summary3" : recom[14],
                   "url4": recom[15], "img4" : recom[16],"title4" : recom[17], "source4": recom[18],"summary4" : recom[19],
                   "url5": recom[20], "img5" : recom[21],"title5" : recom[22], "source5": recom[23],"summary5" : recom[24],})

In [0]:
if __name__ == '__main__':
    app.run(port=5003, debug=False)

 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2019 09:04:14] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:05:02] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:09:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:19:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:23:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:24:04] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:24:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:25:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:25:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:27:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:29:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:29:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:32:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:32:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:32:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/201